In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append("../utils")
import toolbox_ML as tb
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
df=pd.read_csv('../data_sample/ai4i2020.csv')
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [3]:
# rename dataset columns
df.rename(columns = {'Product ID':'Product_ID',
                    'Air temperature [K]':'Air_temperature',
                     'Process temperature [K]':'Process_temperature',
                     'Rotational speed [rpm]':'Rotational_speed',
                     'Torque [Nm]':'Torque',
                     'Tool wear [min]':'Tool_wear',
                     'Machine failure':'Machine_failure',},
          inplace = True)

In [14]:
df.Machine_failure.value_counts()

Machine_failure
0    9661
1     339
Name: count, dtype: int64

In [4]:
df_fallos = df[df['Machine_failure']==1]
df_no_fallos = df[df['Machine_failure']==0]

In [8]:
columnas_fallos=["TWF","HDF","PWF","OSF","RNF"]

In [21]:
df_fallos.info()


<class 'pandas.core.frame.DataFrame'>
Index: 339 entries, 50 to 9974
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UDI                  339 non-null    int64  
 1   Product_ID           339 non-null    object 
 2   Type                 339 non-null    object 
 3   Air_temperature      339 non-null    float64
 4   Process_temperature  339 non-null    float64
 5   Rotational_speed     339 non-null    int64  
 6   Torque               339 non-null    float64
 7   Tool_wear            339 non-null    int64  
 8   Machine_failure      339 non-null    int64  
 9   TWF                  339 non-null    int64  
 10  HDF                  339 non-null    int64  
 11  PWF                  339 non-null    int64  
 12  OSF                  339 non-null    int64  
 13  RNF                  339 non-null    int64  
dtypes: float64(3), int64(9), object(2)
memory usage: 39.7+ KB


In [6]:
df_no_fallos.TWF.value_counts()

TWF
0    9661
Name: count, dtype: int64

In [10]:
for i in columnas_fallos:
    print(f"{df_fallos[i].value_counts()}")

TWF
0    293
1     46
Name: count, dtype: int64
HDF
0    224
1    115
Name: count, dtype: int64
PWF
0    244
1     95
Name: count, dtype: int64
OSF
0    241
1     98
Name: count, dtype: int64
RNF
0    338
1      1
Name: count, dtype: int64


In [12]:
df['Total_Failures'] = df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)

In [13]:
df['Total_Failures'].value_counts()

Total_Failures
0    9652
1     324
2      23
3       1
Name: count, dtype: int64

In [18]:
# Mostrar filas donde Machine_failure == 0 y Total_Failures != 0
anomalies = df[(df['Machine_failure'] == 0) & (df['Total_Failures'] != 0)]
anomalies

,UDI,Product_ID,Type,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Machine_failure,TWF,HDF,PWF,OSF,RNF,Total_Failures
1221,1222,M16081,M,297.0,308.3,1399,46.4,132,0,0,0,0,0,1,1
1302,1303,L48482,L,298.6,309.8,1505,45.7,144,0,0,0,0,0,1,1
1748,1749,H31162,H,298.4,307.7,1626,31.1,166,0,0,0,0,0,1,1
2072,2073,L49252,L,299.6,309.5,1570,35.5,189,0,0,0,0,0,1,1
2559,2560,L49739,L,299.3,309.0,1447,50.4,140,0,0,0,0,0,1,1
3065,3066,M17925,M,300.1,309.2,1687,27.7,95,0,0,0,0,0,1,1
3452,3453,H32866,H,301.6,310.5,1602,32.3,2,0,0,0,0,0,1,1
5471,5472,L52651,L,302.7,312.3,1346,61.2,170,0,0,0,0,0,1,1
5489,5490,L52669,L,302.6,312.1,1499,35.0,215,0,0,0,0,0,1,1
5495,5496,H34909,H,302.9,312.5,1357,55.0,12,0,0,0,0,0,1,1


In [19]:
df_tipos_fallos = df[(df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1) > 0)]

In [20]:
df_tipos_fallos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348 entries, 50 to 9974
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UDI                  348 non-null    int64  
 1   Product_ID           348 non-null    object 
 2   Type                 348 non-null    object 
 3   Air_temperature      348 non-null    float64
 4   Process_temperature  348 non-null    float64
 5   Rotational_speed     348 non-null    int64  
 6   Torque               348 non-null    float64
 7   Tool_wear            348 non-null    int64  
 8   Machine_failure      348 non-null    int64  
 9   TWF                  348 non-null    int64  
 10  HDF                  348 non-null    int64  
 11  PWF                  348 non-null    int64  
 12  OSF                  348 non-null    int64  
 13  RNF                  348 non-null    int64  
 14  Total_Failures       348 non-null    int64  
dtypes: float64(3), int64(10), object(2)
memory 

In [29]:
df_tipos_fallos.Machine_failure.value_counts()

Machine_failure
1    330
0     18
Name: count, dtype: int64

In [28]:
df_tipos_fallos[df_tipos_fallos["Machine_failure"]==0]


,UDI,Product_ID,Type,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Machine_failure,TWF,HDF,PWF,OSF,RNF,Total_Failures
1221,1222,M16081,M,297.0,308.3,1399,46.4,132,0,0,0,0,0,1,1
1302,1303,L48482,L,298.6,309.8,1505,45.7,144,0,0,0,0,0,1,1
1748,1749,H31162,H,298.4,307.7,1626,31.1,166,0,0,0,0,0,1,1
2072,2073,L49252,L,299.6,309.5,1570,35.5,189,0,0,0,0,0,1,1
2559,2560,L49739,L,299.3,309.0,1447,50.4,140,0,0,0,0,0,1,1
3065,3066,M17925,M,300.1,309.2,1687,27.7,95,0,0,0,0,0,1,1
3452,3453,H32866,H,301.6,310.5,1602,32.3,2,0,0,0,0,0,1,1
5471,5472,L52651,L,302.7,312.3,1346,61.2,170,0,0,0,0,0,1,1
5489,5490,L52669,L,302.6,312.1,1499,35.0,215,0,0,0,0,0,1,1
5495,5496,H34909,H,302.9,312.5,1357,55.0,12,0,0,0,0,0,1,1


In [30]:
df_tipos_fallos[df_tipos_fallos[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1) > 1]


,UDI,Product_ID,Type,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Machine_failure,TWF,HDF,PWF,OSF,RNF,Total_Failures
69,70,L47249,L,298.9,309.0,1410,65.7,191,1,0,0,1,1,0,2
1324,1325,M16184,M,298.8,310.1,1243,74.5,194,1,0,0,1,1,0,2
1496,1497,L48676,L,298.0,308.7,1268,69.4,189,1,0,0,1,1,0,2
3611,3612,L50791,L,301.7,310.9,1405,46.4,207,1,1,0,0,0,1,2
3854,3855,L51034,L,302.4,311.0,1338,67.6,194,1,0,0,1,1,0,2
3943,3944,L51123,L,302.3,311.4,1333,66.7,205,1,0,0,1,1,0,2
4254,4255,L51434,L,302.6,311.0,1284,68.0,114,1,0,1,1,0,0,2
4342,4343,M19202,M,301.7,309.8,1284,68.2,111,1,0,1,1,0,0,2
4370,4371,L51550,L,302.0,309.9,1308,57.6,197,1,0,1,0,1,0,2
4383,4384,L51563,L,301.7,309.5,1298,65.5,229,1,0,1,0,1,0,2


In [31]:
for i in columnas_fallos:
    df.loc[df[i] == 1, 'Machine_failure'] = 1

In [32]:
df.Machine_failure.value_counts()

Machine_failure
0    9643
1     357
Name: count, dtype: int64

In [33]:
# Renombrar Columnas para que sea más cómodo trabajar con ellas
df.rename(columns = {'Product ID':'Product_ID',
                    'Air temperature [K]':'Air_temperature',
                     'Process temperature [K]':'Process_temperature',
                     'Rotational speed [rpm]':'Rotational_speed',
                     'Torque [Nm]':'Torque',
                     'Tool wear [min]':'Tool_wear',
                     'Machine failure':'Machine_failure',},
          inplace = True)
# Codificar 'Type' para valores numéricos
df['Type'] = df['Type'].map({'L': 1, 'M': 2, 'H': 3})

# Incluimos todos los fallos en la columa Machine_failure ya que hay fallos que no están contemplados.
columnas_fallos=["TWF","HDF","PWF","OSF","RNF"]
for i in columnas_fallos:
    df.loc[df[i] == 1, 'Machine_failure'] = 1
#Dado que hay una columan de error de pontencia agregamos una nuevacolumna con el valor de la pontencia.
#Esto facilitará la clasficicacion de este error.
df['Power'] = df[['Rotational_speed', 'Torque']].product(axis=1)
# Eliminamos las columnas que no aportan información relevante para el modelo
df.drop(columns=['Product_ID','UDI'], axis=1, inplace=True)
#Eliminamos las columnas de fallos ya que hemos incluido todos los fallos en la columna Machine_failure
df.drop(columns=columnas_fallos, axis=1, inplace=True)
# Separar features y target
X = df.drop(columns=['Machine_failure'])
y = df['Machine_failure']

In [40]:
excluded_columns = ['Machine_failure', 'Type']